In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/ingredients101recipes5k"))
#print(os.listdir("../input/food41"))
from tqdm import tqdm, tqdm_notebook, tnrange

# Any results you write to the current directory are saved as output.

['ingredients101', 'recipes5k']


In [2]:
#all_img_df = pd.DataFrame({'path': glob(os.path.join(BASE_PATH, 'images', '*', '*.jpg'))})
#all_img_df['category'] = all_img_df['path'].map(lambda x: os.path.split(os.path.dirname(x))[-1])
#https://www.kaggle.com/kmader/vgg-on-food-subset-torch/data
# print(os.listdir("../input/food41/images"))
# epochs = 2
# batch_size = 64
# MICRO_DATA = True # very small subset (just 3 groups)
# SAMPLE_TRAINING = False # make train set smaller for faster iteration
# IMG_SIZE = (128, 128)

In [3]:
#Dataframe for train images
# new_data = []
# for index, row in all_img_df.iterrows():
#     #get binary encoding ingredients from lookup
#     food = row["class_name"]
#     path = row["path"]
#     class_id = row["class_id"]
#     binary_encod = food_dict.loc[food_dict["target"] == food]
#     binary_encod["path"] = path
#     binary_encod["class_id"] = int(class_id)
#     #print(binary_encod)
#     #print((list(binary_encod.columns.values)))
#     #print(len(np.array(binary_encod)[0]))
#     new_data.append(np.array(binary_encod)[0])
    

In [4]:
# col_names = list(binary_encod.columns.values)
# train_df = pd.DataFrame(new_data, columns = col_names)

In [5]:
#Dataframe for test images
# new_test_data = []
# for index, row in test_img_df.iterrows():
#     #get binary encoding ingredients from lookup
#     food = row["class_name"]
#     path = row["path"]
#     class_id = row["class_id"]
#     binary_encod = food_dict.loc[food_dict["target"] == food]
#     binary_encod["path"] = path
#     binary_encod["class_id"] = int(class_id)
#     new_test_data.append(np.array(binary_encod)[0])

In [6]:
# test_df = pd.DataFrame(new_test_data, columns = col_names)

In [7]:
# train_df.to_hdf('train_df.h5','df',mode='w',format='table',data_columns=True)
# test_df.to_hdf('test_df.h5','df',mode='w',format='table',data_columns=True)

In [8]:
#START HERE
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/ingredients101recipes5k"))
#print(os.listdir("../input/food41"))
from tqdm import tqdm, tqdm_notebook, tnrange

%matplotlib inline
import os
import sys
import cv2
import numpy as np
import torch
import torch.utils.data as data
from torch.autograd import Variable
import torch.nn as nn
from torchvision import datasets, transforms
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.models as models

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(42) # try and make the results more reproducible
BASE_PATH = '../input/ingredients101recipes5k/ingredients101/Ingredients101/'

print(os.listdir("../input/food41/images"))
epochs = 8
batch_size = 64
SMALL_DATA = False
IMG_SIZE = (384, 384)

['ingredients101', 'recipes5k']
['baklava', 'chicken_wings', 'lobster_roll_sandwich', 'pulled_pork_sandwich', 'edamame', 'tuna_tartare', 'french_toast', 'tacos', 'pad_thai', 'escargots', 'fried_rice', 'beet_salad', 'omelette', 'poutine', 'falafel', 'pork_chop', 'filet_mignon', 'oysters', 'nachos', 'cup_cakes', 'lasagna', 'foie_gras', 'deviled_eggs', 'chicken_quesadilla', 'dumplings', 'ceviche', 'takoyaki', 'scallops', 'pho', 'ice_cream', 'lobster_bisque', 'samosa', 'caprese_salad', 'bread_pudding', 'spaghetti_carbonara', 'risotto', 'red_velvet_cake', 'bruschetta', 'guacamole', 'bibimbap', 'grilled_salmon', 'grilled_cheese_sandwich', 'gyoza', 'ramen', 'creme_brulee', 'steak', 'beef_tartare', 'croque_madame', 'chicken_curry', 'paella', 'breakfast_burrito', 'beignets', 'macarons', 'hot_dog', 'eggs_benedict', 'spaghetti_bolognese', 'macaroni_and_cheese', 'cannoli', 'cheese_plate', 'chocolate_cake', 'club_sandwich', 'prime_rib', 'waffles', 'fish_and_chips', 'sushi', 'pancakes', 'hummus', 'f

In [9]:
print(os.listdir("../input/test-data"))

['test_df.h5']


In [10]:
train_df = pd.read_hdf("../input/preloaded/train_df.h5")
val_df = pd.read_hdf("../input/preloaded/test_df.h5")
test_df = pd.read_hdf("../input/test-data/test_df.h5")

if SMALL_DATA:
    train_df = train_df[:128]
    val_df = test_df[:128]
    test_df = actual_test_df[:128]

In [11]:
col_names = list(train_df.columns.values)

In [12]:
ing_names = col_names[:-3]
targets = ing_names

In [13]:
class DataWrapper(data.Dataset):
    ''' Data wrapper for pytorch's data loader function '''
    def __init__(self, image_df, resize):
        self.dataset = image_df
        self.resize = resize

    def __getitem__(self, index):
        c_row = self.dataset.iloc[index]
        target_arr = []
        for item in c_row[targets].values:
            target_arr.append(item)
        #print(target_arr)
        image_path, target = c_row['path'], torch.from_numpy(np.array(target_arr)).float()  #image and target
        #read as rgb image, resize and convert to range 0 to 1
        image = cv2.imread(image_path, 1)
        if self.resize:
            image = cv2.resize(image, IMG_SIZE)/255.0 
        else:
            image = image/255.0
        image = (torch.from_numpy(image.transpose(2,0,1))).float() #NxCxHxW
        return image, target

    def __len__(self):
        return self.dataset.shape[0] 

In [14]:
model = models.resnet50(pretrained=True)
# #freeze layers
# for param in model.parameters():
#      param.requires_grad = False
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(targets))

ct = 0
for name, child in model.named_children():
    ct += 1
    if ct < 8:
        for name2, params in child.named_parameters():
            params.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:03<00:00, 26122187.65it/s]


In [15]:
import torch.nn.functional as F
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

train_dataset = DataWrapper(train_df, True)
train_loader = torch.utils.data.DataLoader(train_dataset,shuffle=True, batch_size=batch_size, pin_memory=False)

val_dataset = DataWrapper(val_df, True)
val_loader = torch.utils.data.DataLoader(val_dataset,shuffle=True, batch_size=batch_size, pin_memory=False)

test_dataset = DataWrapper(test_df, True)
test_loader = torch.utils.data.DataLoader(test_dataset,shuffle=True, batch_size=batch_size, pin_memory=False)


In [16]:
#TRY LATER
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [17]:
from collections import defaultdict
train_results = defaultdict(list)
train_iter, test_iter, best_acc = 0,0,0
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (10, 10))
ax1.set_title('Train Loss')
ax2.set_title('Train Accuracy')
ax3.set_title('Test Loss')
ax4.set_title('Test Accuracy')

f1_scores = defaultdict(list)

for i in tnrange(epochs, desc='Epochs'):
    print("Epoch ",i)
    ## Train Phase
    #Model switches to train phase
    model.train() 
    
    all_outputs = []
    all_targets = []
    # Running through all mini batches in the dataset
    count, loss_val, correct, total = train_iter, 0, 0, 0
    for img_data, target in tqdm_notebook(train_loader, desc='Training'):    
        img_data, target = img_data.to(device), target.to(device)
        
        output = model(img_data) #FWD prop

        loss = criterion(output, target) #Cross entropy loss
        c_loss = loss.data.item()
        ax1.plot(count, c_loss, 'r.')
        loss_val += c_loss

        optimizer.zero_grad() #Zero out any cached gradients
        loss.backward() #Backward pass
        optimizer.step() #Update the weights

        total_batch = (target.size(0) * target.size(1))
        total += total_batch
        output_data = torch.sigmoid(output)>=0.5
        target_data = (target==1.0)
        for arr1,arr2 in zip(output_data, target_data):
            all_outputs.append(list(arr1.cpu().numpy()))
            all_targets.append(list(arr2.cpu().numpy()))
        c_acc = torch.sum((output_data == target_data.to(device)).to(torch.float)).item()
        ax2.plot(count, c_acc/total_batch, 'r.')
        correct += c_acc
        count +=1
        
    all_outputs = np.array(all_outputs)
    all_targets = np.array(all_targets)
    f1score_samples = f1_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    f1score_macro = f1_score(y_true=all_targets, y_pred=all_outputs, average='macro')
    f1score_weighted = f1_score(y_true=all_targets, y_pred=all_outputs, average='weighted')
    recall = recall_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    prec = precision_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    hamming = hamming_score(y_true=all_targets, y_pred=all_outputs)
    
    f1_scores["samples_train"].append(f1score_samples)
    f1_scores["macro_train"].append(f1score_macro)
    f1_scores["weighted_train"].append(f1score_weighted)
    f1_scores["hamming_train"].append(hamming)
    
    train_loss_val, train_iter, train_acc = loss_val/len(train_loader.dataset), count, correct/float(total)
    
    print("Training loss: ", train_loss_val, " train acc: ",train_acc)    
    ## Test Phase
    
    #Model switches to test phase
    model.eval()
    
    all_outputs = []
    all_targets = []
    #Running through all mini batches in the dataset
    count, correct, total, lost_val = test_iter, 0, 0, 0
    for img_data, target in tqdm_notebook(val_loader, desc='Testing'):
        img_data, target = img_data.to(device), target.to(device)
        output = model(img_data)
        loss = criterion(output, target) #Cross entropy loss
        c_loss = loss.data.item()
        ax3.plot(count, c_loss, 'b.')
        loss_val += c_loss
        #Compute accuracy
        #predicted = output.data.max(1)[1] #get index of max
        total_batch = (target.size(0) * target.size(1))
        total += total_batch
        output_data = torch.sigmoid(output)>=0.5
        target_data = (target==1.0)
        #print("Predictions: ", output_data)
        #print("Actual: ", target_data)
        for arr1,arr2 in zip(output_data, target_data):
            all_outputs.append(list(arr1.cpu().numpy()))
            all_targets.append(list(arr2.cpu().numpy()))
        c_acc = torch.sum((output_data == target_data.to(device)).to(torch.float)).item()
        ax4.plot(count, c_acc/total_batch, 'b.')
        correct += c_acc
        count += 1
    
    #print("Outputs: ", len(all_outputs), " x ", len(all_outputs[0]))
    #print("Targets: ", len(all_targets), " x ", len(all_targets[0]))
    
    #F1 Score
    all_outputs = np.array(all_outputs)
    all_targets = np.array(all_targets)
    f1score_samples = f1_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    f1score_macro = f1_score(y_true=all_targets, y_pred=all_outputs, average='macro')
    f1score_weighted = f1_score(y_true=all_targets, y_pred=all_outputs, average='weighted')
    recall = recall_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    prec = precision_score(y_true=all_targets, y_pred=all_outputs, average='samples')
    hamming = hamming_score(y_true=all_targets, y_pred=all_outputs)
    
    f1_scores["samples_test"].append(f1score_samples)
    f1_scores["macro_test"].append(f1score_macro)
    f1_scores["weighted_test"].append(f1score_weighted)
    f1_scores["hamming_test"].append(hamming)
    
    #Accuracy over entire dataset
    test_acc, test_iter, test_loss_val = correct/float(total), count, loss_val/len(test_loader.dataset)
    print("Test set accuracy: ",test_acc)
    
    train_results['epoch'].append(i)
    train_results['train_loss'].append(train_loss_val)
    train_results['train_acc'].append(train_acc)
    train_results['train_iter'].append(train_iter)
    
    train_results['test_loss'].append(test_loss_val)
    train_results['test_acc'].append(test_acc)
    train_results['test_iter'].append(test_iter)
    
    #Save model with best accuracy
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), 'best_model.pth') 
fig.savefig('train_curves.png')

Epoch  0


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


Training loss:  0.0013551491933905865  train acc:  0.9698243539478765


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Test set accuracy:  0.9754368212151612
Epoch  1


Training loss:  0.0008795812727660105  train acc:  0.9798357750606191


Test set accuracy:  0.9808592489204868
Epoch  2


In [18]:
# print(train_results['train_loss'])
# print(train_results['train_acc'])
# print(train_results['train_iter'])

# print(train_results['test_loss'])
# print(train_results['test_acc'])
# print(train_results['test_iter'])
print("TRAIN")
print("F1 Samples: ", f1_scores["samples_train"])
print("F1 Weighted: ", f1_scores["weighted_train"])
print("Hamming: ", f1_scores["hamming_train"])
print()
print("==============")
print("VALIDATION")
print("F1 Samples: ", f1_scores["samples_test"])
print("F1 Weighted: ", f1_scores["weighted_test"])
print("Hamming: ", f1_scores["hamming_test"])

TRAIN
F1 Samples:  [0.39065842934129547, 0.61747843594592, 0.7152974466229176, 0.7875360552474681, 0.8489450550486578, 0.8937793057227219, 0.9204696386159933, 0.9362687185862372]
F1 Weighted:  [0.4450719892052109, 0.683555548147284, 0.7770047972621169, 0.8414998701968172, 0.8934099154883484, 0.9283031706091955, 0.9469570482696279, 0.9572537801875907]
Hamming:  [0.3092180999979363, 0.5442021716144451, 0.6542953873040989, 0.737113390865783, 0.8090559376030957, 0.8631689997320106, 0.8958885893529938, 0.9154911601047653]

VALIDATION
F1 Samples:  [0.5542499681017867, 0.6571158407632353, 0.6787851615173598, 0.7091542213199802, 0.7011058375244261, 0.7006272559723696, 0.7072922489039039, 0.7150334819962425]
F1 Weighted:  [0.6057540436259837, 0.7137707999226118, 0.7315812681267617, 0.7536685512201822, 0.7468917884747844, 0.7420260588659202, 0.7476821832518884, 0.7566858150221474]
Hamming:  [0.47100659938420836, 0.5909207853654295, 0.6166885815690424, 0.6500451019611048, 0.6466751627984413, 0.64

In [19]:
#Inference on test
model_path = "best_model.pth"
model.load_state_dict(torch.load(model_path))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

#Run predictions
all_outputs = []
all_targets = []
for img_data, target in tqdm_notebook(test_loader, desc='Testing'):
    img_data, target = img_data.to(device), target.to(device)
    output = model(img_data)
    loss = criterion(output, target) #Cross entropy loss
    c_loss = loss.data.item()
    ax3.plot(count, c_loss, 'b.')
    loss_val += c_loss
    #Compute accuracy
    #predicted = output.data.max(1)[1] #get index of max
    total_batch = (target.size(0) * target.size(1))
    total += total_batch
    output_data = torch.sigmoid(output)>=0.5
    target_data = (target==1.0)
    #print("Predictions: ", output_data)
    #print("Actual: ", target_data)
    for arr1,arr2 in zip(output_data, target_data):
        all_outputs.append(list(arr1.cpu().numpy()))
        all_targets.append(list(arr2.cpu().numpy()))
    c_acc = torch.sum((output_data == target_data.to(device)).to(torch.float)).item()
    ax4.plot(count, c_acc/total_batch, 'b.')
    correct += c_acc
    count += 1


#F1 Score
all_outputs = np.array(all_outputs)
all_targets = np.array(all_targets)
f1score_samples = f1_score(y_true=all_targets, y_pred=all_outputs, average='samples')
f1score_macro = f1_score(y_true=all_targets, y_pred=all_outputs, average='macro')
f1score_weighted = f1_score(y_true=all_targets, y_pred=all_outputs, average='weighted')
recall = recall_score(y_true=all_targets, y_pred=all_outputs, average='samples')
prec = precision_score(y_true=all_targets, y_pred=all_outputs, average='samples')
hamming = hamming_score(y_true=all_targets, y_pred=all_outputs)

In [20]:
print("TEST")
print("F1 Samples: ", f1score_samples)
print("F1 Weighted: ", f1score_weighted)
print("Hamming: ", hamming)

TEST
F1 Samples:  0.7475997578017275
F1 Weighted:  0.7857211724880503
Hamming:  0.6925506689963147
